In [ ]:
#
import tkinter as tk
from tkinter import messagebox
import bitget.v1.mix.order_api as maxOrderApi
import bitget.v1.mix.market_api as market_api
import bitget.v1.mix.account_api as account_api
import time
import bitget.bitget_api as bitget_api
from datetime import datetime
from bitget.exceptions import BitgetAPIException
import tkinter as tk
from tkinter import ttk
import sys
import tkinter as tk
import threading
import time


def show_error_message(error_msg):
    root = tk.Tk()
    root.withdraw()
    messagebox.showerror("Fehler", error_msg)


if __name__ == '__main__':

    apiKey = "xxxx"
    secretKey = '''xxxx'''
    passphrase = "xxxx"

    try:
        maxOrderApi = maxOrderApi.OrderApi(apiKey, secretKey, passphrase)
        market_api = market_api.MarketApi(apiKey, secretKey, passphrase)
        account_api = account_api.AccountApi(apiKey, secretKey, passphrase)
        bitget_api = bitget_api.BitgetApi(apiKey, secretKey, passphrase)

        print("Verbunden")
    except BitgetAPIException as e:
        print("error: ", e)

Verbunden


In [2]:
root = tk.Tk()

In [3]:
class Runden:

    def __init__(self):
        self.Mal_Lot = 2
        self.gradean = False
        self.weitere_runden = None
        self.Werte_in_sich = False
        self.TabridArray = []
        self.LotArray = []
        self.available = 0
        self.client_id = []  # die erste position ist keine order
        # (USDT-M-Futures)            umcbl  = BTCUSDT_UMCBL       ETHUSDT_UMCBL       / quoteCoin = USDT     /supportMarginCoins = USDT
        # (USDT-M-Futures Demo)       sumcbl = SBTCSUSDT_SUMCBL    SETHSUSDT_SUMCBL     /quoteCoin = SUSDT
        self.symbol = "SETHSUSDT_SUMCBL"
        self.productType = "sumcbl"
        self.marginCoin = "SUSDT"
        self.symbolArray = []
        self.unrealizedPL = 0
        self.Unterschied = None
        self.Profit_USD = 0
        self.Max_Order = None
        self.first_Lot = None
        self.Richtung = None
        self.start_Preis = None

    def reset_attributes(self):
        self.available = 0
        self.LotArray = []
        self.Werte_in_sich = False
        self.TabridArray = []
        self.client_id = []
        self.unrealizedPL = 0
        self.start_Preis = None

    def reset_Class(self):
        self.available = 0
        self.gradean = False
        self.Mal_Lot = 2
        self.weitere_runden = None
        self.Werte_in_sich = False
        self.TabridArray = []
        self.LotArray = []
        self.client_id = []  # die erste position ist keine order
        self.symbol = "SETHSUSDT_SUMCBL"
        self.productType = "sumcbl"
        self.marginCoin = "SUSDT"
        self.symbolArray = []
        self.unrealizedPL = 0
        self.Unterschied = None
        self.Profit_USD = 0
        self.Max_Order = None
        self.first_Lot = None
        self.Richtung = None
        self.start_Preis = None

    def print_attributes(self):
        print("TabridArray:", self.TabridArray)
        print("LotArray:", self.LotArray)
        print("client_id:", self.client_id)
        print("symbol:", self.symbol)
        print("symbolArray:", self.symbolArray)
        print("productType:", self.productType)
        print("marginCoin:", self.marginCoin)
        print("unrealizedPL:", self.unrealizedPL)
        print("Unterschied:", self.Unterschied)
        print("Profit_USD:", self.Profit_USD)
        print("Max_Order:", self.Max_Order)
        print("first_Lot:", self.first_Lot)
        print("Richtung:", self.Richtung)
        print("start_Preis:", self.start_Preis)

    def AlleSymboleAnzeigen(self):
        params = {}
        params["productType"] = self.productType
        response = market_api.contracts(params)

        for item in response['data']:
            self.symbolArray.append(item['symbol'])

    def Rechnen_GV(self):
        try:
            params = {}
            params["productType"] = self.productType
            params["symbol"] = self.symbol
            response = bitget_api.get(
                "/api/mix/v1/position/allPosition-v2", params)
            for item in response['data']:
                if item['symbol'] == self.symbol:
                    self.unrealizedPL = float(item['unrealizedPL'])
                    self.available = float(item['available'])
        except BitgetAPIException as e:
            print("Rechnen_GV(self)")

    def close_all_positions(self):

        if (self.Richtung == "open_long"):
            side = "close_long"
        else:
            side = "close_short"
        try:
            params = {}
            params["symbol"] = self.symbol
            params["marginCoin"] = self.marginCoin
            params["side"] = side
            params["orderType"] = "market"
            params["size"] = self.available
            params["timInForceValue"] = "normal"
            response = maxOrderApi.placeOrder(params)
        except BitgetAPIException as e:
            print("close_all_positions(self)")
            print("Keine Position Geschlösen:")

    def get_market_price(self):
        params = {"symbol": self.symbol}
        data = market_api.ticker(params)
        market_price = data['data']['last']
        return market_price

    def lot_rechnen(self):
        self.LotArray.append(round(self.first_Lot, 2))
        for i in range(1, self.Max_Order):
            prev_sum = sum(self.LotArray[:i])
            ergebnis = prev_sum * self.Mal_Lot

            self.LotArray.append(round(ergebnis, 2))

    def Order_in_array(self):
        self.TabridArray.append(self.start_Preis)
        if self.Richtung == "open_long":
            for i in range(1, self.Max_Order):
                new_tabrid_value = self.TabridArray[i-1] - self.Unterschied
                self.TabridArray.append(new_tabrid_value)

        elif self.Richtung == "open_short":
            for i in range(1, self.Max_Order):
                new_tabrid_value = self.TabridArray[i-1] + self.Unterschied
                self.TabridArray.append(new_tabrid_value)

    def Order_Send_limit(self, preis, lot):
        try:
            params = {}
            params["symbol"] = self.symbol
            params["marginCoin"] = self.marginCoin
            params["side"] = self.Richtung
            params["orderType"] = "limit"
            params["price"] = preis
            params["size"] = lot
            params["timInForceValue"] = "normal"
            response = maxOrderApi.placeOrder(params)
            response[0] = response
            self.client_id.append(int(response[0]['data']['clientOid']))
        except Exception as e:
            error_msg = "Order kann nicht gesendet werden (zu viel lot?)"
            show_error_message(error_msg)

    def Market_Order_Position(self):
        try:
            params = {}
            params["symbol"] = self.symbol
            params["marginCoin"] = self.marginCoin
            params["side"] = self.Richtung
            params["orderType"] = "market"
            params["size"] = self.first_Lot
            params["timInForceValue"] = "normal"
            response = maxOrderApi.placeOrder(params)

        except Exception as e:
            error_msg = "Roboter ist Aus Jetzt \nMarket_Order_Position (zu viel lot ?)"
            show_error_message(error_msg)
            root.destroy()
            sys.exit()

        try:
            params = {}
            params["productType"] = runde1.productType
            params["symbol"] = runde1.symbol
            data = bitget_api.get(
                "/api/mix/v1/position/allPosition", params)
        except BitgetAPIException as e:
            error_msg = "Get Data Market_Order_Position()"
            show_error_message(error_msg)

            # print("2 Market_Order_Position(self)")
            # print("error:" + e.message)

        gesuchtes_symbol = self.symbol
        gesuchte_hold_side = 'long'  # oder 'short'
        if (self.Richtung == "open_long"):
            gesuchte_hold_side = 'long'
        elif (self.Richtung == "open_short"):
            gesuchte_hold_side = 'short'

        for item in data['data']:
            if item['symbol'] == gesuchtes_symbol and item['holdSide'] == gesuchte_hold_side:
                average_open_price_str = item['averageOpenPrice']
                average_open_price_float = float(
                    average_open_price_str)
                self.start_Preis = round(average_open_price_float, 2)
                break

    def Order_Send_to_Bitget(self):

        for i in range(1, self.Max_Order):
            self.Order_Send_limit(
                preis=self.TabridArray[i], lot=self.LotArray[i])

    def Order_Loeschen(self):
        try:
            for client_oid in self.client_id:
                params = {}
                params["symbol"] = self.symbol
                params["marginCoin"] = self.marginCoin
                params["clientOid"] = client_oid
                gg = maxOrderApi.cancelOrder(params)
        except BitgetAPIException as e:
            print("Keine Order für Löschen ?")

    def neue_Runde(self):
        try:
            self.gradean = True
            self.Market_Order_Position()
            self.lot_rechnen()
            self.Order_in_array()
            self.Order_Send_to_Bitget()
            # self.print_attributes()
        except Exception as e:
            error_msg = "Gneue_Runde"
            show_error_message(error_msg)
            # print("Ein Fehler ist aufgetreten in Market_Order_Position():", e)
            self.reset_attributes()
            self.reset_Class()

            # Hier könnten Sie weitere Aktionen ausführen, je nach Bedarf

    def Alles_schliessen(self):
        self.close_all_positions()
        self.Order_Loeschen()
        self.reset_attributes()


runde1 = Runden()
runde2 = Runden()
runde1.AlleSymboleAnzeigen()
runde2.AlleSymboleAnzeigen()


response :  {"code":"00000","msg":"success","requestTime":1707782486725,"data":[{"symbol":"SBTCSUSDT_SUMCBL","makerFeeRate":"0.0002","takerFeeRate":"0.0006","feeRateUpRatio":"0.1","openCostUpRatio":"0.1","quoteCoin":"SUSDT","baseCoin":"SBTC","buyLimitPriceRatio":"0.01","sellLimitPriceRatio":"0.01","supportMarginCoins":["SUSDT"],"minTradeNum":"0.001","priceEndStep":"1","volumePlace":"3","pricePlace":"1","sizeMultiplier":"0.001","symbolType":"perpetual","symbolStatus":"normal","offTime":"-1","limitOpenTime":"-1","maintainTime":"","symbolName":"SBTCSUSDT","minTradeUSDT":null,"maxPositionNum":null,"maxOrderNum":null},{"symbol":"SETHSUSDT_SUMCBL","makerFeeRate":"0.0002","takerFeeRate":"0.0006","feeRateUpRatio":"0.1","openCostUpRatio":"0.1","quoteCoin":"SUSDT","baseCoin":"SETH","buyLimitPriceRatio":"0.03","sellLimitPriceRatio":"0.03","supportMarginCoins":["SUSDT"],"minTradeNum":"0.001","priceEndStep":"1","volumePlace":"3","pricePlace":"2","sizeMultiplier":"0.001","symbolType":"perpetual","sy

In [4]:
"""
(USDT-M-Futures)            umcbl  = BTCUSDT_UMCBL       ETHUSDT_UMCBL       / quoteCoin = USDT     /supportMarginCoins = USDT
(Coin-M Perptual-Futeres)   dmcbl  = BTCUSD_DMCBL        ETHUSD_DMCBL        /quoteCoin = USD       /supportMarginCoins = ['BTC', 'ETH', 'USDC', 'XRP', 'BGB', 'STETH']
(USDC-M-Futures)            cmcbl  = BTCPERP_CMCBL       ETHPERP_CMCBL       /quoteCoin = USD       /supportMarginCoins = USDC
(USDT-M-Futures Demo)       sumcbl = SBTCSUSDT_SUMCBL    SETHSUSDT_SUMCBL     /quoteCoin = SUSDT
(Coin-M Futeres Demo)       sdmcbl = SBTCSUSD_SDMCBL     SETHSUSD_SDMCBL     /quoteCoin = SUSD
(USDC-M-Futures Demo)       scmcbl = SBTCSPERP_SCMCBL    SETHSPERP_SCMCBL    /quoteCoin = SUSD
"""

"\n(USDT-M-Futures)            umcbl  = BTCUSDT_UMCBL       ETHUSDT_UMCBL       / quoteCoin = USDT     /supportMarginCoins = USDT\n(Coin-M Perptual-Futeres)   dmcbl  = BTCUSD_DMCBL        ETHUSD_DMCBL        /quoteCoin = USD       /supportMarginCoins = ['BTC', 'ETH', 'USDC', 'XRP', 'BGB', 'STETH']\n(USDC-M-Futures)            cmcbl  = BTCPERP_CMCBL       ETHPERP_CMCBL       /quoteCoin = USD       /supportMarginCoins = USDC\n(USDT-M-Futures Demo)       sumcbl = SBTCSUSDT_SUMCBL    SETHSUSDT_SUMCBL     /quoteCoin = SUSDT\n(Coin-M Futeres Demo)       sdmcbl = SBTCSUSD_SDMCBL     SETHSUSD_SDMCBL     /quoteCoin = SUSD\n(USDC-M-Futures Demo)       scmcbl = SBTCSPERP_SCMCBL    SETHSPERP_SCMCBL    /quoteCoin = SUSD\n"

In [5]:
class schleife:
    schleife_1 = True
    schleife_2 = True

In [6]:
e = 25
L = 20


def flasch_Button():
    try:
        runde1.Alles_schliessen()
        runde2.Alles_schliessen()
        runde1.reset_Class()
        runde2.reset_Class()
        Rund1_label_Anzeigen()
        Rund2_label_Anzeigen()

    except Exception as e:
        error_msg = str(e)
        show_error_message(error_msg)


def Ende2_Button():
    if (runde2.weitere_runden == True):
        runde2.weitere_runden = False
        ende2_button.config(bg="red")
        Rund1_label_Anzeigen()
        Rund2_label_Anzeigen()


def Ende1_Button():
    if (runde1.weitere_runden == True):
        runde1.weitere_runden = False
        ende1_button.config(bg="red")
        Rund1_label_Anzeigen()
        Rund2_label_Anzeigen()

def schleife_runde2():
    while schleife.schleife_2:
        try:
            runde2.Rechnen_GV()
            Rund1_label_Anzeigen()
            Rund2_label_Anzeigen()
        except Exception as e:
            Rund1_label_Anzeigen()
            Rund2_label_Anzeigen()

            error_msg = "runde2.Rechnen_GV() " + str(e)
            show_error_message(error_msg)
            schleife.schleife_2 = False

        if (runde2.unrealizedPL >= runde2.Profit_USD):
            runde2.Alles_schliessen()
            if (runde2.weitere_runden == True):
                runde2.neue_Runde()
            else:
                runde2.reset_Class()
                ende2_button.config(bg="white")
                save_button.config(bg="green")
                Rund1_label_Anzeigen()
                Rund2_label_Anzeigen()
                if (runde1.weitere_runden == True and runde1.Werte_in_sich == True):
                    runde1.neue_Runde()
                    t1 = None
                    schleife.schleife_1 = True
                    t1 = threading.Thread(target=schleife_runde1)
                    t1.start()
                else:
                    root.destroy()
                    sys.exit()
                schleife.schleife_2 = False
        time.sleep(0.5)


def schleife_runde1():
    while schleife.schleife_1:
        try:
            runde1.Rechnen_GV()
            Rund1_label_Anzeigen()
            Rund2_label_Anzeigen()
        except Exception as e:
            Rund1_label_Anzeigen()
            Rund2_label_Anzeigen()
            error_msg = "runde1.Rechnen_GV() " + str(e)
            show_error_message(error_msg)
            schleife.schleife_1 = False

        if (runde1.unrealizedPL >= runde1.Profit_USD):
            runde1.Alles_schliessen()
            if (runde1.weitere_runden == True):
                runde1.neue_Runde()
            else:
                runde1.reset_Class()
                ende1_button.config(bg="white")
                save_button.config(bg="green")
                Rund1_label_Anzeigen()
                Rund2_label_Anzeigen()
                if (runde2.weitere_runden == True and runde2.Werte_in_sich == True):
                    runde2.neue_Runde()
                    t2 = None
                    schleife.schleife_2 = True
                    t2 = threading.Thread(target=schleife_runde2)
                    t2.start()
                else:
                    print("schließen")
                    # root.destroy()
                    # sys.exit()
                schleife.schleife_1 = False
        time.sleep(0.5)


def start_Button():
    if (runde1.Werte_in_sich == False):
        runde1.Werte_in_sich = True
        try:
            runde1.weitere_runden = True
            runde1.symbol = symbol_var.get()
            runde1.Unterschied = float(Tabrid_entry.get())
            runde1.first_Lot = float(Lot_entry.get())
            runde1.Mal_Lot = float(Mal_Lot_entry.get())
            runde1.Max_Order = int(Max_Order_entry.get())
            runde1.Profit_USD = float(ProfitUSD_entry.get())
            Richtung = Richtung_var.get()

            if Richtung == "Buy":
                runde1.Richtung = "open_long"
            elif Richtung == "Sell":
                runde1.Richtung = "open_short"
            else:
                raise ValueError("Ungültige Richtung")

            try:
                Rund1_label_Anzeigen()
                Rund2_label_Anzeigen()

                if (runde1.Werte_in_sich == False or runde2.Werte_in_sich == False):
                    save_button.config(bg="green")
                elif (runde1.Werte_in_sich == True and runde2.Werte_in_sich == True):
                    save_button.config(bg="red")

                if (runde2.gradean == False):
                    runde1.neue_Runde()
                    t = threading.Thread(target=schleife_runde1)
                    t.start()

            except Exception as e:
                error_msg = "1" + str(e)
                show_error_message(error_msg)
                # root.destroy()
                # sys.exit()

        except Exception as e:
            runde1.reset_Class()
            error_msg = "Falsche Eingabe " + str(e)
            show_error_message(error_msg)
            # root.destroy()
            # sys.exit()

    elif (runde1.Werte_in_sich == True and runde2.Werte_in_sich == False):
        runde2.Werte_in_sich = True

        try:
            runde2.weitere_runden = True
            runde2.symbol = symbol_var.get()
            runde2.Unterschied = float(Tabrid_entry.get())
            runde2.first_Lot = float(Lot_entry.get())
            runde2.Mal_Lot = float(Mal_Lot_entry.get())
            runde2.Max_Order = int(Max_Order_entry.get())
            runde2.Profit_USD = float(ProfitUSD_entry.get())
            Richtung = Richtung_var.get()
            if Richtung == "Buy":
                runde2.Richtung = "open_long"
            elif Richtung == "Sell":
                runde2.Richtung = "open_short"

        except Exception as e:
            runde2.reset_Class()
            error_msg = "Falsche Eingabe " + str(e)
            show_error_message(error_msg)
            # root.destroy()
            # sys.exit()

        if (runde1.Werte_in_sich == False or runde2.Werte_in_sich == False):
            save_button.config(bg="green")
        elif (runde1.Werte_in_sich == True and runde2.Werte_in_sich == True):
            save_button.config(bg="red")

        Rund1_label_Anzeigen()
        Rund2_label_Anzeigen()

    else:
        show_error_message("Keine Platz für Runde")
        # root.destroy()
        # sys.exit()


def Rund1_label_Anzeigen():
    Rund1_label.config(
        text=("Runde 1: \n" +
              "Grade An: " + str(runde1.gradean) + "\n" +
              "UnrealizedPL: " + str(runde1.unrealizedPL) + "\n" +
              "Nächte Runde: " + str(runde1.weitere_runden) + "\n" +
              "Werte_in_sich: " + str(runde1.Werte_in_sich) + "\n" +
              "available: " + str(runde1.available) + "\n" +
              "Symbol: " + str(runde1.symbol) + "\n" +
              "Tabrid: " + str(runde1.Unterschied) + "\n" +
              "Lot: " + str(runde1.first_Lot) + "\n" +
              "Mal : " + str(runde1.Mal_Lot) + "\n" +
              "Max Order: " + str(runde1.Max_Order) + "\n" +
              "Profit USD: " + str(runde1.Profit_USD) + "\n" +
              "Richtung: " + str(runde1.Richtung)
              ), justify="left"
    )


def Rund2_label_Anzeigen():
    Rund2_label.config(
        text=("Runde 2: \n" +
              "Grade An: " + str(runde2.gradean) + "\n" +
              "UnrealizedPL: " + str(runde2.unrealizedPL) + "\n" +
              "Nächte Runde: " + str(runde2.weitere_runden) + "\n" +
              "Werte_in_sich: " + str(runde2.Werte_in_sich) + "\n" +
              "available: " + str(runde2.available) + "\n" +
              "Symbol: " + str(runde2.symbol) + "\n" +
              "Tabrid: " + str(runde2.Unterschied) + "\n" +
              "Lot: " + str(runde2.first_Lot) + "\n" +
              "Mal : " + str(runde2.Mal_Lot) + "\n" +
              "Max Order: " + str(runde2.Max_Order) + "\n" +
              "Profit USD: " + str(runde2.Profit_USD) + "\n" +
              "Richtung: " + str(runde2.Richtung)
              ), justify="left"
    )


def Runde1_Entfernen():
    if (runde1.gradean == False):
        runde1.reset_Class()
        runde1.reset_attributes()
        save_button.config(bg="green")

        Rund1_label_Anzeigen()
        Rund2_label_Anzeigen()


def Runde2_Entfernen():
    if (runde2.gradean == False):
        runde2.reset_Class()
        runde2.reset_attributes()
        save_button.config(bg="green")
        Rund1_label_Anzeigen()
        Rund2_label_Anzeigen()


def on_closing():
    print("Das Fenster wird geschlossen")
    root.destroy()


def GuiZeigen():

    global Tabrid_entry, Runde2_Entfernen_button, Runde1_Entfernen_button, Lot_entry, Max_Order_entry, ProfitUSD_entry, Richtung_var, Mal_Lot_entry, save_button, ende1_button, ende2_button, flasch_button, symbol_var, Rund1_label, Rund2_label
    root.title("Robot 1")
    window_width = 1200
    window_height = 900

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    x_position = (screen_width // 2) - (window_width // 2)
    y_position = (screen_height // 2) - (window_height // 2)

    root.geometry(f"{window_width}x{window_height}+{x_position}+{y_position}")

    leer_label = tk.Label(
        root, text="                                             ", font=("Helvetica", L))
    leer_label.grid(row=0, column=0, sticky="w")

    leer_label = tk.Label(
        root, text="                                             ", font=("Helvetica", L))
    leer_label.grid(row=0, column=1, sticky="w")

    symbol_label = tk.Label(root, text="Symbol:", font=("Helvetica", L))
    symbol_label.grid(row=1, column=0, sticky="w")
    symbol_var = tk.StringVar()
    symbol_combobox = ttk.Combobox(
        root, textvariable=symbol_var, font=("Helvetica", e), width=14)
    symbol_combobox['values'] = runde1.symbolArray
    symbol_combobox.grid(row=1, column=1, sticky="ew")
    symbol_combobox.insert(0, runde1.symbol)

    Tabrid_label = tk.Label(root, text="Tabrid:", font=("Helvetica", L))
    Tabrid_label.grid(row=2, column=0, sticky="w")
    Tabrid_entry = tk.Entry(root, font=("Helvetica", e), width=15)
    Tabrid_entry.grid(row=2, column=1, sticky="ew")
    Tabrid_entry.insert(0, "20")

    Lot_label = tk.Label(root, text="First Lot:", font=("Helvetica", L))
    Lot_label.grid(row=3, column=0, sticky="w")
    Lot_entry = tk.Entry(root, font=("Helvetica", e), width=15)
    Lot_entry.grid(row=3, column=1, sticky="ew")
    Lot_entry.insert(0, "0.1")

    Mal_Lot_label = tk.Label(root, text="Mal ?:", font=("Helvetica", L))
    Mal_Lot_label.grid(row=4, column=0, sticky="w")
    Mal_Lot_entry = tk.Entry(root, font=("Helvetica", e), width=15)
    Mal_Lot_entry.grid(row=4, column=1, sticky="ew")
    Mal_Lot_entry.insert(0, "2")

    Max_Order_label = tk.Label(root, text="Max Order:", font=("Helvetica", L))
    Max_Order_label.grid(row=5, column=0, sticky="w")
    Max_Order_entry = tk.Entry(root, font=("Helvetica", e), width=15)
    Max_Order_entry.grid(row=5, column=1, sticky="ew")
    Max_Order_entry.insert(0, "4")

    ProfitUSD_label = tk.Label(root, text="Profit USD:", font=("Helvetica", L))
    ProfitUSD_label.grid(row=6, column=0, sticky="w")
    ProfitUSD_entry = tk.Entry(root, font=("Helvetica", e), width=15)
    ProfitUSD_entry.grid(row=6, column=1, sticky="ew")
    ProfitUSD_entry.insert(0, "10")

    Richtung_label = tk.Label(root, text="Richtung:", font=("Helvetica", L))
    Richtung_label.grid(row=7, column=0, sticky="w")
    Richtung_var = tk.StringVar()
    Richtung_combobox = ttk.Combobox(
        root, textvariable=Richtung_var, font=("Helvetica", e), width=14)
    Richtung_combobox['values'] = ('Buy', 'Sell')
    Richtung_combobox.grid(row=7, column=1, sticky="ew")

    save_button = tk.Button(root, text="      حفظ      ", font=(
        "Helvetica", L), command=start_Button)
    save_button.grid(row=8, column=0)
    save_button.config(bg="green")

    ende1_button = tk.Button(root, text="    1 ايقاف تداول    ", font=(
        "Helvetica", L), command=Ende1_Button)
    ende1_button.grid(row=8, column=1, pady=5)
    ende1_button.config(bg="white")

    ende2_button = tk.Button(root, text="    2 ايقاف تداول    ", font=(
        "Helvetica", L), command=Ende2_Button)
    ende2_button.grid(row=9, column=1, pady=5)
    ende2_button.config(bg="white")

    flasch_button = tk.Button(root, text=" SOFOT Alles AUS ", font=(
        "Helvetica", L), command=flasch_Button)
    flasch_button.grid(row=8, column=3)
    flasch_button.config(bg="red")

    Rund1_label = tk.Label(root, text="Runde 1:", font=("Helvetica", L))
    Rund1_label.grid(row=9, column=0, sticky="w")

    Rund2_label = tk.Label(root, text="Runde 2:", font=("Helvetica", L))
    Rund2_label.grid(row=9, column=3, sticky="w")

    root.protocol("WM_DELETE_WINDOW", on_closing)

    Runde1_Entfernen_button = tk.Button(root, text="      مسح      ", font=(
        "Helvetica", L), command=Runde1_Entfernen)
    Runde1_Entfernen_button.grid(row=10, column=0)
    Runde1_Entfernen_button.config(bg="white")

    Runde2_Entfernen_button = tk.Button(root, text="      مسح      ", font=(
        "Helvetica", L), command=Runde2_Entfernen)
    Runde2_Entfernen_button.grid(row=10, column=3)
    Runde2_Entfernen_button.config(bg="white")

    Rund1_label_Anzeigen()
    Rund2_label_Anzeigen()

    root.mainloop()


GuiZeigen()


response :  {"code":"40762","msg":"委托数量大于最大可开数量","requestTime":1707782491180,"data":null}
status: 400


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
"""
(USDT-M-Futures Demo)       sumcbl = SBTCSUSDT_SUMCBL    SETHSUSDT_SUMCBL     /quoteCoin = SUSDT
(Coin-M Futeres Demo)       sdmcbl = SBTCSUSD_SDMCBL     SETHSUSD_SDMCBL     /quoteCoin = SUSD
(USDC-M-Futures Demo)       scmcbl = SBTCSPERP_SCMCBL    SETHSPERP_SCMCBL    /quoteCoin = SUSD
"""

'\n(USDT-M-Futures Demo)       sumcbl = SBTCSUSDT_SUMCBL    SETHSUSDT_SUMCBL     /quoteCoin = SUSDT\n(Coin-M Futeres Demo)       sdmcbl = SBTCSUSD_SDMCBL     SETHSUSD_SDMCBL     /quoteCoin = SUSD\n(USDC-M-Futures Demo)       scmcbl = SBTCSPERP_SCMCBL    SETHSPERP_SCMCBL    /quoteCoin = SUSD\n'